# Computing distance/similarity matrices for hypothesis testin

In [1]:
import datetime
import time
from itertools import combinations
from pathlib import Path

from graspologic.embed import *
from joblib import Parallel, delayed
from pkg.data import load_dataset
from pkg.inference import difference_norm
from scipy.spatial.distance import squareform

In [2]:
df = pd.read_csv("../restricted_data/690subjects.csv")

subjects = df.Subject.astype(int).values

## Computing Connectome Distances

In [3]:
def compute_zg(subjects, graphs, workers=81):
    n_verts = graphs[str(subjects[0])].shape[0]

    n_dims = int(np.sqrt(n_verts) * 2)
    zg = Parallel(n_jobs=workers)(
        delayed(select_dimension)(g, n_dims) for g in graphs.values()
    )
    max_zg = np.max([d[0][-1] for d in zg])

    return max_zg

In [4]:
def compute_connectome_distances(subjects, graphs, n_components=None, workers=81):
    """
    subjects : list
    graphs : dict
    """
    subs = [str(s) for s in subjects]

    if n_components is not None:
        n_verts = graphs[subs[0]].shape[0]
        n_dims = n_verts // 2

        if n_dims > 100:  # for computational efficiency
            n_dims = 100

        zg = Parallel(n_jobs=workers)(
            delayed(select_dimension)(g, n_dims) for g in graphs.values()
        )
        max_zg = np.max([d[0][-1] for d in zg])
        n_components = max_zg

    def worker(pairs, n_components):
        out = []
        for u, v in pairs:
            G = graphs[u]
            H = graphs[v]
            X = AdjacencySpectralEmbed(
                n_components=n_components, check_lcc=False
            ).fit_transform(G)
            Y = AdjacencySpectralEmbed(
                n_components=n_components, check_lcc=False
            ).fit_transform(H)
            res = [
                difference_norm(X, Y, model) for model in ["exact", "global", "vertex"]
            ]
            out.append(res)
        return np.array(out)

    cartesian_prods = list(combinations(subs, 2))
    splits = np.array_split(cartesian_prods, workers)

    distances = Parallel(n_jobs=workers)(
        delayed(worker)(split, n_components) for split in splits
    )

    distances = np.vstack(distances)

    return distances

In [5]:
parcellations = [
    "AAL",
    "CPAC200",
    "DKT",
    "Desikan",
    "Glasser",
    "Yeo-17-liberal",
    "Yeo-17",
    "Yeo-7-liberal",
    "Yeo-7",
    "Schaefer200",
    "Schaefer300",
    "Schaefer400",
    "Schaefer1000",
]

In [6]:
zg = {"AAL": 10}

In [10]:
t0 = time.time()

for parcellation in parcellations:
    print(f"Computing {parcellation}")
    graphs = load_dataset(
        parcellation=parcellation,
    )
    graphs = {key: val for key, val in graphs.items() if int(key) in subjects}

    if parcellation not in zg.keys():
        zg[parcellation] = compute_zg(subjects, graphs)

    connectome_distances = compute_connectome_distances(
        subjects, graphs, n_components=zg[parcellation]
    )

    np.savez_compressed(
        f"../results/{parcellation}/connectome_distances",
        exact=connectome_distances[:, 0],
        glob=connectome_distances[:, 1],
        vertex=connectome_distances[:, 2],
        subjects=subjects,
    )

Computing AAL
Computing CPAC200
Computing DKT
Computing Desikan
Computing Glasser
Computing Yeo-17-liberal
Computing Yeo-17
Computing Yeo-7-liberal
Computing Yeo-7
Computing Schaefer200
Computing Schaefer300
Computing Schaefer400
Computing Schaefer1000


In [11]:
elapsed = time.time() - t0
delta = datetime.timedelta(seconds=elapsed)
print(f"Script took {delta}")

Script took 1:23:57.927655


In [12]:
zg

{'AAL': 10,
 'CPAC200': 11,
 'DKT': 8,
 'Desikan': 4,
 'Glasser': 9,
 'Yeo-17-liberal': 6,
 'Yeo-17': 6,
 'Yeo-7-liberal': 4,
 'Yeo-7': 4,
 'Schaefer200': 14,
 'Schaefer300': 16,
 'Schaefer400': 17,
 'Schaefer1000': 27}

## Compute genomic distance

Only need to do this once since it is constant regardless of parcellation choice

In [ ]:
def compute_genome_distances(subjects, df, workers=81):
    """
    subjects : list
    df : pd.DataFrame

    0 = MZ, 1 = NotTwin/DZ, 2 = non-twin sibling, 3 = step-sibling, 4 = unrelated.
    """

    def worker(pairs, df):
        rel_dict = dict(MZ=0, DZ=1, NotTwin=2, Step=3, Unrelated=4)

        dists = []
        for u, v in pairs:
            subs = df[df.Subject.isin([u, v])]
            fdx1, mdx1, fdx2, mdx2 = subs[["Father_ID", "Mother_ID"]].values.ravel()
            zyg1, zyg2 = subs[["Zygosity"]].values.ravel()

            # below checks if same family
            if (fdx1 == fdx2) and (mdx1 == mdx2):
                if zyg1 == zyg2:
                    dist = rel_dict[zyg1]
                elif zyg1 != zyg2:
                    dist = rel_dict["NotTwin"]
            elif (fdx1 != fdx2) ^ (mdx1 != mdx2):  # share at least one parent
                dist = rel_dict["Step"]
            elif (fdx1 != fdx2) and (mdx1 != mdx2):  # no sharing parents
                dist = rel_dict["Unrelated"]
            else:  # sanity check
                raise ValueError()
            dists.append(dist)
        return dists

    subs = [int(s) for s in subjects]
    cartesian_prods = list(combinations(subs, 2))
    splits = np.array_split(cartesian_prods, workers)
    distances = Parallel(n_jobs=workers)(delayed(worker)(split, df) for split in splits)

    return np.hstack(distances)

In [44]:
genome = compute_genome_distances(subjects, df)

np.savez_compressed("../results/genome_distance", genome=genome, subjects=subjects)

## Compute neuroanatomy distance matrix and full covariate similarity matrix

In [95]:
parcs = [
    "AAL_space-MNI152NLin6_res-1x1x1",
    "CPAC200_space-MNI152NLin6_res-1x1x1",
    "DKT_space-MNI152NLin6_res-1x1x1",
    "Desikan_space-MNI152NLin6_res-1x1x1",
    "Glasser_space-MNI152NLin6_res-1x1x1",
    "Schaefer1000_space-MNI152NLin6_res-1x1x1",
    "Schaefer200_space-MNI152NLin6_res-1x1x1",
    "Schaefer300_space-MNI152NLin6_res-1x1x1",
    "Schaefer400_space-MNI152NLin6_res-1x1x1",
    "Yeo-17-liberal_space-MNI152NLin6_res-1x1x1",
    "Yeo-17_space-MNI152NLin6_res-1x1x1",
    "Yeo-7-liberal_space-MNI152NLin6_res-1x1x1",
    "Yeo-7_space-MNI152NLin6_res-1x1x1",
]

In [103]:
ages = np.array([df[df.Subject == sub].Age_in_Yrs.values for sub in subjects]).reshape(
    -1, 1
)
ages = ages / ages.max()


for parc in parcs:
    pname = parc.split("_")[0]
    print(f"Computing {pname}")

    files = [cov_path / f"{sub}_{parc}.csv.csv" for sub in subjects]
    covariates = [pd.read_csv(f) for f in files]

    res = []
    for covariate in covariates:
        covariate = covariate[["Volume", "FA", "AD", "RD"]]
        covariate = np.nan_to_num(covariate)
        volume = covariate[:, 0]
        rest = covariate[:, 1:]

        normed = rest * volume.reshape(-1, 1)

        to_append = [volume.mean(), *rest.mean(axis=0)]
        res.append(to_append)

    res = np.array(res)
    res = res / res.max(axis=0)

    cols = ["Volume", "FA", "AD", "RD"]
    dists = pdist(res)

    np.savez_compressed(
        f"../results/{pname}/covariate_distance",
        neuroanatomy=dists,
        subjects=subjects,
    )

    res = np.hstack([res, ages])
    cols = ["Volume", "FA", "AD", "RD", "Age"]
    cd = ConditionalDcorr()
    simmat = cd._compute_kde(res)

    np.savez_compressed(
        f"../results/{pname}/covariate_kde",
        covariates=simmat,
        subjects=subjects,
    )